In [1]:
import tensorflow as tf
import keras
import keras_nlp

EMBEDDING_DIM = 64
NUM_HEADS = 2
INTERMIDIATE_DIM = 256

dataset = [
    (
        "Giant pig fell into the swimming pool at his home in Ringwood, Hampshire. It took the efforts of a team of firefighters to winch him out of the water. A wayward horse also had to be rescued from a swimming pool in Sussex.",
        "<start> Giant pig fell into the swimming pool.",
        "Giant pig fell into the swimming pool. <end>",
    ),
    (
        "There are two chickens in the garden.",
        "<start> There are chickens.",
        "There are chickens. <end>",
    ),
]

# Preprocessing
input_texts, target_texts, decoder_target_text = zip(*dataset)

tokenizer = keras.preprocessing.text.Tokenizer(
    split=' ',
    filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
)
tokenizer.fit_on_texts(input_texts + target_texts + decoder_target_text)
vocab_size = len(tokenizer.word_index) + 1

input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)
decoder_target_sequences = tokenizer.texts_to_sequences(decoder_target_text)

max_input_length = max(len(seq) for seq in input_sequences)
max_target_length = max(len(seq) for seq in target_sequences)

input_sequences = keras.preprocessing.sequence.pad_sequences(
    input_sequences,
    maxlen=max_input_length,
    padding='post'
)
target_sequences = keras.preprocessing.sequence.pad_sequences(
    target_sequences,
    maxlen=max_target_length,
    padding='post'
)
decoder_target_sequences = keras.preprocessing.sequence.pad_sequences(
    decoder_target_sequences,
    maxlen=max_target_length,
    padding='post'
)
decoder_target_sequences = tf.expand_dims(decoder_target_sequences, axis=-1)

# Build model / Encoder & Decoder model
encoder_inputs = keras.Input(
    shape=(max_input_length,),
    name="encoder_inputs"
)
encoder_embedding = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=vocab_size,
    sequence_length=max_input_length,
    embedding_dim=EMBEDDING_DIM,
)(encoder_inputs)
encoder_outputs = keras_nlp.layers.TransformerEncoder(
    num_heads=NUM_HEADS,
    intermediate_dim=INTERMIDIATE_DIM,
)(encoder_embedding)

decoder_inputs = keras.Input(
    shape=(max_target_length,),
    name="decoder_inputs"
)
decoder_embedding = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=vocab_size,
    sequence_length=max_target_length,
    embedding_dim=EMBEDDING_DIM,
)(decoder_inputs)
decoder_outputs = keras_nlp.layers.TransformerDecoder(
    num_heads=NUM_HEADS,
    intermediate_dim=INTERMIDIATE_DIM,
)(decoder_embedding, encoder_outputs)

outputs = keras.layers.Dense(
    vocab_size,
    activation="softmax"
)(decoder_outputs)

model = keras.Model(
    [encoder_inputs, decoder_inputs],
    outputs
)
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Training
model.fit(
    [input_sequences, target_sequences],
    decoder_target_sequences,
    epochs=100,
)


Using TensorFlow backend


2024-06-04 10:13:41.865912: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-06-04 10:13:41.865935: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-06-04 10:13:41.865940: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-06-04 10:13:41.865971: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-04 10:13:41.865990: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/100


2024-06-04 10:13:44.807645: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 3s 3s/step - loss: 4.4164 - accuracy: 0.0625
Epoch 2/100
1/1 [==============================] - 0s 47ms/step - loss: 3.3720 - accuracy: 0.1250
Epoch 3/100
1/1 [==============================] - 0s 43ms/step - loss: 2.4921 - accuracy: 0.1875
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 1.8311 - accuracy: 0.5625
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.3532 - accuracy: 0.6875
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 0.9908 - accuracy: 0.9375
Epoch 7/100
1/1 [==============================] - 0s 51ms/step - loss: 0.7163 - accuracy: 0.9375
Epoch 8/100
1/1 [==============================] - 0s 35ms/step - loss: 0.5202 - accuracy: 0.9375
Epoch 9/100
1/1 [==============================] - 0s 33ms/step - loss: 0.3870 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 0.2969 - accuracy: 1.0000
Epoch 11/100
1/1 [===============

In [2]:
def summarize(text):
    """
    Summarize text
    :param text: original text
    :return: summarized text
    """
    input_sequence = tokenizer.texts_to_sequences([text])
    input_sequence = keras.preprocessing.sequence.pad_sequences(
        input_sequence,
        maxlen=max_input_length,
        padding='post'
    )
    idx = tokenizer.word_index['<start>']
    decoder_input_sequence = tf.constant(
        [[idx]],
        dtype=tf.int64
    )
    
    summary = []
    for _ in range(max_target_length):
        predictions = model.predict(
            [input_sequence, decoder_input_sequence],
            verbose=0
        )
        next_token = tf.argmax(predictions[0, -1, :])
        next_word = tokenizer.index_word.get(next_token.numpy(), '<unk>')
        if next_word == '<end>':
            break
        summary.append(next_word)
        decoder_input_sequence = tf.concat(
            [decoder_input_sequence, tf.expand_dims([next_token], axis=-1)],
            axis=-1
        )
    return ' '.join(summary)

# Sample
sample_text = "Giant pig fell into the swimming pool at his home in Ringwood, Hampshire. It took the efforts of a team of firefighters to winch him out of the water. A wayward horse also had to be rescued from a swimming pool in Sussex."
print("Original text:", sample_text)
print("Summary:", summarize(sample_text))

sample_text = "There are two chickens in the garden."
print("Original text:", sample_text)
print("Summary:", summarize(sample_text))

sample_text = "Two chickens fell into the swimming pool in the garden."
print("Original text:", sample_text)
print("Summary:", summarize(sample_text))

Original text: Giant pig fell into the swimming pool at his home in Ringwood, Hampshire. It took the efforts of a team of firefighters to winch him out of the water. A wayward horse also had to be rescued from a swimming pool in Sussex.
Summary: giant pig fell into the swimming pool
Original text: There are two chickens in the garden.
Summary: there are chickens
Original text: Two chickens fell into the swimming pool in the garden.
Summary: there are chickens


In [3]:
tokenizer.word_index

{'the': 1,
 'swimming': 2,
 'pool': 3,
 'giant': 4,
 'pig': 5,
 'fell': 6,
 'into': 7,
 'in': 8,
 'of': 9,
 'a': 10,
 'there': 11,
 'are': 12,
 'chickens': 13,
 'to': 14,
 '<start>': 15,
 '<end>': 16,
 'at': 17,
 'his': 18,
 'home': 19,
 'ringwood': 20,
 'hampshire': 21,
 'it': 22,
 'took': 23,
 'efforts': 24,
 'team': 25,
 'firefighters': 26,
 'winch': 27,
 'him': 28,
 'out': 29,
 'water': 30,
 'wayward': 31,
 'horse': 32,
 'also': 33,
 'had': 34,
 'be': 35,
 'rescued': 36,
 'from': 37,
 'sussex': 38,
 'two': 39,
 'garden': 40}